In [1]:
import random, string, math, re, os
import fitz
# from pyhanlp import HanLP
from faker import Faker

In [2]:
import cv2
import matplotlib.pyplot as plt

In [4]:
import numpy as np

In [6]:
import pytesseract 
from PIL import Image, ImageDraw, ImageSequence, ImageFont
# import en_core_web_lg

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to C:\Users\jacky
[nltk_data]     tam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jacky tam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\jacky
[nltk_data]     tam\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\jacky
[nltk_data]     tam\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [8]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

## Basic Setting

In [9]:
doc_text = []
fake = Faker("zh_tw")
doc = ''
colorList = fitz.utils.getColorList()
font_language = {1:'zh', 2:'en'}

#### Read file

In [10]:
def readFile(path):
    global doc, doc_text
    pdf_path = path
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()
        doc_text.append(text)
    return doc_text

#### Fake Name Generator

In [11]:
def conditional_fake(cond):
    while True:
        x = fake.name()
        if cond(x):
            yield x

## Chinese Parser

#### LAC

In [12]:
from LAC import LAC
lac = LAC(mode="lac")    

In [13]:
def extract_LAC(sentence):
    user_name_lis = []
    _result = lac.run(sentence)
    print(_result)
    for _index, _label in enumerate(_result[1]):
        if _label == "PER":
            user_name_lis.append(_result[0][_index])
    return user_name_lis

#### LTP

In [14]:
from ltp import LTP
ltp = LTP("LTP/small")  # 默认加载 Small 模型

G:\Anaconda\main\envs\data_masking\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def extract_LTP(sentence):
    user_name_list = []
    try:
        output = ltp.pipeline([sentence], tasks=["cws", "pos", "ner", "srl", "dep", "sdp"])
        # 使用字典格式作为返回结果
#         print(output.cws)  # print(output[0]) / print(output['cws']) # 也可以使用下标访问
#         print(output.pos)
#         print(output.sdp)
        pos_length = len(output.pos[0])
        i = 0
        while i < pos_length:
            if(pos_length - i)>=3:
                temp = '-'.join(output.pos[0][i:i+3])
                # search for pattern 'nh', 'nh', 'n'
                if(temp=='nh-nh-n' or temp=='o-nh-ws'):
                    user_name_list.append(''.join(output.cws[0][i:i+3]))
                    i += 3
                    continue

            if(pos_length - i)>=2:
                temp = '-'.join(output.pos[0][i:i+2])

                if(temp=='nh-nh'):
                    # search for pattern nh + nh 
                    user_name_list.append(''.join(output.cws[0][i:i+2]))
                    i += 2
                    continue

                elif(temp=='nh-n'):
                    # search for pattern nh + n
                    user_name_list.append(''.join(output.cws[0][i:i+2]))
                    i += 2
                    continue
            # search for pattern nh
            if(output.pos[0][i] == 'nh'):
                user_name_list.append(output.cws[0][i])
            i += 1
    except Exception as e:
        print(text, e)
    finally:
        return user_name_list

#         elif type == 'ltp':
#             output = ltp.pipeline([sentence], tasks=["cws", "pos", "ner", "srl", "dep", "sdp"])
#             # 使用字典格式作为返回结果
#             print(output.cws)  # print(output[0]) / print(output['cws']) # 也可以使用下标访问
#             print(output.pos)
#             print(output.sdp)
#             print(" ")
#             for i in range(len(output.pos)):
#                 if(output.pos[i] == 'nh'):
#                     user_name_lis.append(output.cws[i])
#     #  1.   2.   3. 
    # [['丁', '小姐']]
    # [['nh', 'n']]
    # [{'head': [2, 0], 'label': ['FEAT', 'Root']}]
    # [['陳大文']]
    # [['nh']]
    # [{'head': [0], 'label': ['Root']}]

#### JIEBA

In [16]:
import jieba
import jieba.posseg as pseg

In [17]:
def extract_jieba(text):
#     words = pseg.cut(text) #jieba默认模式
    # jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
#     words = pseg.cut(text,use_paddle=True) #paddle模式
#     for word, flag in words:
#         print('%s %s' % (word, flag))
    per_list = []  # 人名列表
    word_list = jieba.lcut(text)
#     print(word_list)
    for word in word_list:
        if len(word)==1:  # 不加判断会爆
            continue
        words = pseg.cut(word, use_paddle=True)  # paddle模式
#         print(list(words))
        word, flag = list(words)[0]
        if flag=='PER':  # 这里写成LOC是地名
            per_list.append(word)
    per_list = list(set(per_list))
#     print(per_list)
    return per_list

#### Extraction Model Handler

In [18]:
def extract_name(sentence: str, type='lac'):
    user_name_lis = []
#     print(sentence)
    try:
        if type == 'lac':
            lacResult = extract_LAC(sentence)
            if len(lacResult)==0:
                ltpResult = extract_LTP(sentence)
                user_name_lis.extend(ltpResult)
            user_name_lis.extend(lacResult)
        elif type == 'ltp':
            ltpResult = extract_LTP(sentence)
            user_name_lis.extend(ltpResult)
        elif type == 'jieba':
            jiebaResult = extract_jieba(sentence)
            user_name_lis.extend(jiebaResult)
        else:
            raise Exception('type not suppose')
        return user_name_lis
    except Exception as e:
        print(text, e)
    finally:
        return user_name_lis

In [19]:
def searchNameMatch(_text, opt='lac'):
    nameExtracted = extract_name(_text,opt)
    return nameExtracted

#### Find Word Distance (to find font file)

In [20]:
# find word distance
import Levenshtein

def getFontPath(font_name, lang = 'zh'):
    base_path = 'c:/windows/fonts'
    end_path = ''
    fileName_search = font_name.lower()
    fileNames = []
    for fileName in os.listdir(base_path):
        fileName_inList = fileName.replace(" ","").replace(r'_-',"").lower().split('.')
        # print(fileName_inList)
        if((Levenshtein.distance(fileName_search, fileName_inList[0])<5 
            or (fileName_inList[0] in fileName_search))):
#            and fileName_inList[1]=='ttf'):
            fileNames.append(fileName)
    if(len(fileNames)>0):
        base_path = base_path+'/'+fileNames[0]
    else:
        if not (lang=='zh'):
            base_path +='/times.ttf'
        else:
            base_path +='/kaiu.ttf'
    return base_path

## Search ID

In [21]:
def searchIDMatch(_text):
#     print(_text)
#     reg = r"([a-zA-Z][0-9]{6})\s*\(\s*[0-9]\s*\)"
    reg = r"[a-zA-Z]{0,1}[0-9]{6,7}\s*\([0-9]{0,1}\){0,1}"
    trackID = re.findall(reg,_text)
    if(trackID):
        return(trackID)
    return None

In [22]:
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

## Rewrite the text

In [37]:
def rewriteName(name_tuple, _blockText, _page, imgflag = True, _imgConfig = None):
    nameExtracted = name_tuple[0]
    newName = name_tuple[1]
    nameLang = name_tuple[2]

#         if(nameExtracted):
        # print(nameExtracted)
        ##
        # get text format
#                 print(blockText)
    fontSize, fontType, fontColor, fontdir = 1,'arial',0,(0,0),
    text_loca = [[0,0,0,0]]
    fontColorRGB = (0,0,0)
    
    if imgflag:
        fontSize = _blockText['lines'][0]['spans'][0]['size']
        fontType = _blockText['lines'][0]['spans'][0]['font']
        fontColor = _blockText['lines'][0]['spans'][0]['color']
        fontdir = _blockText['lines'][0]['dir']
        text_loca = _page.search_for(nameExtracted)
        fontColorRGB = fitz.utils.getColor(colorList[fontColor].lower())
    else:
        fontSize = _imgConfig["fontSize"]
        text_loca = _imgConfig["text_loca"]
        fontType = _imgConfig["fontType"]if _imgConfig["fontType"] != None else fontType
        fontColor = _imgConfig["fontColor"] if _imgConfig["fontColor"] != None else fontColor
        fontdir = _imgConfig["fontdir"] if _imgConfig["fontdir"] != None else fontdir
        
    
    fontRotationDegree = math.atan2(fontdir[0], fontdir[1])*180/math.pi - _page.rotation
    fontFilePath = getFontPath(fontType,font_language[nameLang])

    print('---> Name Change')
    print(f'{nameExtracted} -> {newName}')
    print(f'Location: {text_loca[0]} | Font Size: {fontSize} | Font Type: {fontType} | Font Path: {fontFilePath}')
    print(f'Color: {fontColor} -> {fontColorRGB} : {colorList[fontColor]} | Rotation: {fontdir} -> {fontRotationDegree} | Page Rotation: {_page.rotation}')
    ##
    # define font 
    _page.draw_rect(
        text_loca[0],
        color=(1,1,1),
        fill=(1,1,1),
        width=0
    )
    _page.insert_font(fontname=fontType,fontfile=fontFilePath, fontbuffer=newName , set_simple=False )
    _page.insert_text(
        fitz.Point(text_loca[0][2],text_loca[0][3]),
        newName,
        fontname=fontType,
        fontsize=fontSize,
        lineheight=1,
#         color=(0.5,0.5,0),
        color=fontColorRGB, 
        # fill=(1,1,1,1), 
        render_mode=0, 
        border_width=1, 
        rotate=fontRotationDegree, 
        morph=None, 
        overlay=True
    )
    _page.draw_rect(
        text_loca[0],
        color=(1,1,0.6),
        fill=(1,1,0.6),
        fill_opacity=0.5,
        width=0
    )
    print(" ")

## Process PDF 

In [24]:
def process_pdf(_page,i):
    textPage = _page.get_textpage().extractDICT()
    for blockText in textPage['blocks']:
#             text = blockText['lines'][0]['spans'][0]['text']
        modifyList = []
        for textLine in blockText['lines']:
            for textSpan in textLine['spans']:
                text = textSpan['text']

                list_id_found = searchIDMatch(text)
                if(list_id_found):
                    for idd in list_id_found:
                        randomID = id_generator(1,string.ascii_uppercase) + id_generator(6,string.digits) + "("+id_generator(1,string.digits)+")"
                        modifyList.append([idd, randomID,2])
                #ltp
                if(len(text)>0 and (text.count(" ") != len(text))):
#                         list_name_found = searchNameMatch(text,'ltp')
#                         if(list_name_found):
#                             for _name in list_name_found:
#                                 modifyList.append([_name, fake.name(),1])
                #lac
                    list_name_found = searchNameMatch(text,'lac')
                    if(list_name_found):
                        for _name in list_name_found:
                            # generate fake naem
                            zhWordCheck = re.findall(r'[\u4e00-\u9fff]+', _name)
                            if zhWordCheck:
                                fake = Faker("zh_tw")
                            else:
                                fake = Faker("en")
                            modifyList.append([_name, fake.name(),1])
#                         list_name_found = searchNameMatch(text,'ltp')
#                         print(list_name_found)
#                         print('-------------')
        if(len(modifyList)>0):
            for obj in modifyList:
                rewriteName(obj,blockText,_page)

In [65]:
def process_scannedpdf(_page, i):
    try:
        # pre process image
        img_name = '' + str(i) + '.png'
        rotate = int(0)
        zoom = 1.5
        trans = fitz.Matrix(zoom, zoom).prerotate(rotate)
        scale_pixmap = _page.get_pixmap(matrix=trans, alpha=False,colorspace=fitz.csRGB)
        img = Image.frombytes("RGB", [scale_pixmap.width, scale_pixmap.height], scale_pixmap.samples)
        img = img.convert('L')

        cleaned_page_array = cv2.adaptiveThreshold(np.array(img), 
                                255,
                                # cv2.ADAPTIVE_THRESH_MEAN_C,
                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY,
                                27,
                                20)
        new_image = Image.fromarray(cleaned_page_array)
        new_image.save(img_name)
        
        # plt.figure(figsize = (100,100))
        # plt.imshow(cleaned_page_array)
        
        eng_text = pytesseract.image_to_string(new_image,"eng")
        d = pytesseract.image_to_data(new_image, output_type=pytesseract.Output.DICT)
#         d_df = pd.DataFrame.from_dict(d)
#         print(eng_text)
        eng_text_split = eng_text.split('\n')
        # search for words
        modifyList = []
        for text in eng_text_split:
            list_id_found = searchIDMatch(text)
            if(list_id_found):
                for idd in list_id_found:
                    randomID = id_generator(1,string.ascii_uppercase) + id_generator(6,string.digits) + "("+id_generator(1,string.digits)+")"
                    modifyList.append([idd, randomID,2])
        # write text
        if(len(modifyList)>0):
#             print("?")
            for obj in modifyList:
                # calculate text box
                n_boxes = len(d['text'])

                obj_split = obj[0].split(" ")
                cur_id = 0
                cood_list = []
                for i in range(n_boxes):
        #             print(i)
        #             print(d['text'][i]+"+++++"+str(d['conf'][i])+"+++++"+obj_split[cur_id])
        #             print(obj_split[cur_id] in d['text'][i])
                    if (obj_split[cur_id] in d['text'][i]  and (d['conf'][i] >20)):

                        cood_list.append([d['left'][i], d['top'][i], d['width'][i], d['height'][i],d['conf'][i]])
                        cur_id +=1
        #                 print(cood_list, i)
                        while cur_id < len(obj_split):
                            if(i+cur_id < n_boxes):
                                if (obj_split[cur_id] in d['text'][i] and d['conf'][i] >20):
                                    cood_list.append([d['left'][i+cur_id], d['top'][i+cur_id], d['width'][i+cur_id], d['height'][i+cur_id],d['conf'][i+cur_id]])
                                    cur_id +=1
        #                             print(cood_list, i)
                                else:
                                    break
                                    cood_list = []
                                    cur_id = 0
                    if len(cood_list) == len(obj_split):
                        break
        #         print(cood_list)
                x_0 = cood_list[0][0]
                y_0 = cood_list[0][1]
                x_1 = cood_list[-1][0] + cood_list[-1][2]
                y_1 = cood_list[-1][1] + cood_list[-1][3]

                textBox_config = {
                    'text_loca':[[x_0,y_0,x_1,y_1]],
                    "fontSize":int((y_1 - y_0)/16*12),
                    "fontType":None,
                    "fontColor":None,
                    "fontdir":None
                }
        #         print(textBox_config)
        #         textBox_config["text_loca"] = 
        #         textBox_config[] = 

                rewriteName(obj,None,_page, False, textBox_config)
        os.remove(img_name)
    except Exception as e:
        print(e)

### Search Each Page

In [26]:
def writeText(fileType='PDF'):
    global fake, doc
    for i in range(len(doc)):
    # for i in range(2):
        print(f">>Current editing page {i} ")
        page = doc[i]
        if not page.is_wrapped:
            page.wrap_contents() 
        if fileType=="PDF":
            process_pdf(page,i)
        elif fileType=="IMAGEPDF":
            process_scannedpdf(page,i)


## Process Image File

## Main Workflow

In [52]:
def workflow(inPath, outPath, inName,outName, typeDoc):
    global doc, doc_text
    doc = ''
    print("Step1: reading documents")
    fileread = ''
    try:
        fileread = readFile(inPath + inName)
    except IOError:
        print("Error: file not found")
    else:
        print('text read successfully')
    
#     print(fileread)
    
    print("**************************************************")
    print("Step2: modify documents")
    try:
        writeText(typeDoc)
    except Exception as e:
        print('Error: write text failed')
        print(e)
    else:
        print('text changed successfully')
    
    print("**************************************************")
    print("Step3: save documents")
    try:
        doc.save(outPath + outName)
    except Exception as e:
        print('Error: file saved failed')
        print(e)
    else:
        print('file saved successfully')

# Starting Point

In [183]:
## Initiate Workflow
workflow('', '', '1.pdf', "1_out3.pdf", "PDF")

Step1: reading documents
text changed successfully
**************************************************
Step2: modify documents
>>Current editing page 0 
[['丁小姐'], ['PER']]
[['9888', ' ', '8888', ' '], ['m', 'w', 'm', 'w']]
---> Name Change
丁小姐 -> 蔣雅慧
Location: Rect(233.42977905273438, 459.8136291503906, 241.40977478027344, 483.80950927734375) | Font Size: 7.980000019073486 | Font Type: PMingLiU | Font Path: c:/windows/fonts/mingliu.ttc
Color: 154 -> (0.10196078431372549, 0.10196078431372549, 0.10196078431372549) : GRAY10 | Rotation: (0.0, -1.0) -> 90.0 | Page Rotation: 90
 
[['xyz@abc.com', ' '], ['nw', 'w']]
[['01/12/2012', ' '], ['nw', 'w']]
[['31', '/', '12', '/', '2012', ' '], ['m', 'w', 'q', 'w', 'TIME', 'w']]
[[' ', '0', ' ', '0  0', ' '], ['w', 'm', 'w', 'm', 'w']]
[['0 0 0  1  2  3   4', ' '], ['', 'w']]
[['甲乙丙公司'], ['ORG']]
[['5', ' ', '6', ' ', '7', ' ', '8', ' '], ['m', 'w', 'm', 'w', 'm', 'w', 'm', 'w']]
[['李積金'], ['']]
[['A123456', ' ', '(', '7', ')', ' '], ['n', 'w', 'w', 

In [66]:
workflow('', '', '6.pdf', "6_out.pdf", "IMAGEPDF")

Step1: reading documents
text read successfully
**************************************************
Step2: modify documents
>>Current editing page 0 
---> Name Change
X112233(0) -> B675893(7)
Location: [315, 319, 376, 341] | Font Size: 16 | Font Type: arial | Font Path: c:/windows/fonts/ALGER.TTF
Color: 0 -> (0.9411764705882353, 0.9725490196078431, 1.0) : ALICEBLUE | Rotation: (0, 0) -> 0.0 | Page Rotation: 0
 
text changed successfully
**************************************************
Step3: save documents
file saved successfully


In [ ]:
#             page.insert_text(
#                 point=fitz.Point(text_loca[0][0],text_loca[0][2]),
#                 text=fake.name(),
#                 fontsize=10,
# #                 fontname=fontType,
#                 fontfile=,
#                 color=(0,0,0),
#                 lineheight=12
#             )           
#             print(text_loca)
            # write text
#             page.add_redact_annot(
#                 quad=text_loca[0],
#                 text=fake.name(),
#                 fontname=fontType,
#                 fontfile=getFontPath(fontType),
#                 fontsize=fontSize,
#                 text_color=(1,1,1),
#                 cross_out=False
#             )
#             page.apply_redactions()

        # insert the text
#                 page.insert_text(
#                     fitz.Point(text_loca[0][2],text_loca[0][3]),
#                     nameExtracted,
#                     fontname=fontType,
#                     fontsize=fontSize,
#                     lineheight=1,
#                     color=fontColorRGB, 
#                     fill=(1,1,1,1), 
#                     render_mode=0, 
#                     border_width=1, 
#                     rotate=fontRotationDegree, 
#                     morph=None, 
#                     overlay=True
#                 )

    # _page.insert_textbox(
    #     text_loca[0],
    #     newName,
    #     fontname=fontType,
    #     fontsize=fontSize,
    #     lineheight=2,
    #     color=fontColorRGB, 
    #     fill=(1,1,1,1), 
    #     render_mode=0, 
    #     border_width=1, 
    #     # rotate=fontRotationDegree, 
    #     rotate=0, 
    #     morph=None, 
    #     overlay=False
    # )

Building prefix dict from the default dictionary ...
Loading model from cache G:\TMPTMP\jieba.cache
Loading model cost 0.674 seconds.
Prefix dict has been built successfully.


我 r
爱 v
北京 ns
天安门 ns
